# Cargar Modelo

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import cv2
import math
from sklearn.preprocessing import normalize

import keras
from keras import preprocessing
from keras import layers
from keras.models import Sequential



C:\Users\Yulisa\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


# Load Model

In [2]:
new_model = keras.models.load_model('cnn_model_keras2.h5')
new_model.summary()

OSError: Unable to open file (unable to open file: name = 'cnn_model_keras2.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

# Clasificar imagen

In [16]:
class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0','1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

def translate_prediction(prediction):
    if(len(class_names) >= int(prediction)):
        return class_names[int(prediction)]
    return ""

def test_modelo(image):
    img_height = 50
    img_width = 50

    img = cv2.resize(image, (img_width, img_height))
    img = np.array(img, dtype=np.float32)    
    cv2.imwrite(os.path.join("" , "reshaped.jpg"), img)
    img = np.reshape(img, (1, img_width, img_height, 1))
    predictions = new_model.predict(img)[0]

#     score = predictions / np.linalg.norm(predictions)
    score = normalize(predictions[:,np.newaxis], axis=0).ravel()
    prediction = np.argmax(score)
    print(len(predictions), prediction )
    return "This image most likely belongs to {} (index: {}) with a {:.2f} percent confidence.".format(translate_prediction(prediction), prediction, 100 * np.max(score))
    

# Capturar Imagen

In [6]:
#Init Webcam
cap = cv2.VideoCapture(0)
result = True

while result:
    obtain, frame = cap.read()

    # Simulating mirror image
    frame = cv2.flip(frame, 1)
        

    # Coordinates of the Region of Interest
    x1 = int(0.5*frame.shape[1])
    y1 = 10
    x2 = frame.shape[1]-10
    y2 = int(0.5*frame.shape[1])
    # Drawing the Region of Interest
    # The increment/decrement by 1 is to compensate for the bounding box
    cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255,0,0) ,1)
    # Extracting the Region of Interest
    roi = frame[y1:y2, x1:x2]
    frame = frame[y1:y2, x1:x2]
    #Para cambiar el tamaño de la ventana del roi modificar los ultimos dos numeros
    roi = cv2.resize(roi, (500, 500)) 
    #frame = cv2.resize(frame, (1024,1024))
     
    #mostrar camara normal
    cv2.imshow("Frame", frame)
        
    #GaussianBlur para eliminar ruido    
    roi = cv2.GaussianBlur(roi, (5,5),0)
    #threshold previous capture image
    _, roi = cv2.threshold(roi, 120, 255, cv2.THRESH_BINARY)
    
    #MedianBlur para suavizar contorno
    i = 0
    while i < 15:
       roi = cv2.medianBlur(roi, 7);
       i+=1  
    
    # do the processing after capturing the image!
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    
    #Mostrar ROI
#     roi = cv2.bitwise_not(roi)
    cv2.imshow("ROI", roi)
    

    if cv2.waitKey(1) == ord('c'):
        #Image with no filter
#         pathNoFilter = 'C:/Users/Yulisa/Desktop/Imagenes/SinFiltro'
        image_name = "image_sample.jpg"
#         cv2.imwrite(os.path.join(pathNoFilter , image_name), frame[y1:y2, x1:x2])
#         print("Foto guardada sin filtro {}".format(image_name))

        #Image with filter
        cv2.imwrite(os.path.join("" , image_name), roi)
#         print("Foto guardada con filtro {}".format(image_name))
        print(test_modelo(roi))
        
        result = True

    interrupt = cv2.waitKey(1)
    if interrupt & 0xFF == 27: # esc key
        result = False

cap.release()
cv2.destroyAllWindows()